In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [3]:
! pip install -U sentence-transformers accelerate datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 25.5 MB/s eta 0:00:00


In [2]:
import random
import pandas as pd
from sentence_transformers import InputExample
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
from sentence_transformers import losses

In [3]:
OUT_DIR = "/content/drive/MyDrive/rag"
CSV_PATH = f"{OUT_DIR}/rag_finetune_dataset.csv"
OUTPUT_PATH = "/content/drive/MyDrive/rag/bge-m3-rag-finetuned"

EMBED_MODEL = "BAAI/bge-m3"
EPOCHS = 1
BATCH_SIZE = 2

In [6]:
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=["question", "context"])

train_set = []
for _, row in df.iterrows():
    train_set.append(
        InputExample(
            texts=[row["question"], row["context"]],
            label=1.0
        )
    )

contexts = df["context"].tolist()
for _, row in df.iterrows():
    neg_context = random.choice(contexts)
    if neg_context != row["context"]:
        train_set.append(
            InputExample(
                texts=[row["question"], neg_context],
                label=0.0
            )
        )

print("Train examples:", len(train_set))

Train examples: 10357


In [7]:
train_dataloader = DataLoader(
    train_set,
    shuffle=True,
    batch_size=BATCH_SIZE
)

In [8]:
embed_model = SentenceTransformer(EMBED_MODEL, device='cuda')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [9]:
train_loss = losses.CosineSimilarityLoss(embed_model)

embed_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=100,
    show_progress_bar=True,
    use_amp=True,
    output_path=OUTPUT_PATH
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


Step,Training Loss
500,0.145400
1000,0.126000
1500,0.111300
2000,0.092200
2500,0.099900
3000,0.098200
3500,0.096900
4000,0.083700
4500,0.085400
5000,0.076300


In [10]:
from google.colab import runtime
runtime.unassign()